<a href="https://colab.research.google.com/github/youngmoo/ECES-435/blob/main/Class6-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-435: Class 6.2**

**Announcements**
* No lab this week (enjoy the break!)
* For next class, please watch Video No. 9: *The z-Transform*.



# Optional stuff

## Interactive Matplotlib

Install `ipympl` for  Matplotlib


In [ ]:
#!pip install ipympl   # Also installs a more recent version of matplotlib (v3.5.3)

Enable interactive Matplotlib figures

In [ ]:
# from google.colab import output
# output.enable_custom_widget_manager()
# %matplotlib widget

## My plot style defaults

In [ ]:
from matplotlib import rc

rc('figure', figsize=(12,4))
rc('figure', facecolor='#aaaaaa')     # Better figure background for dark mode

rc('font', family='Liberation Serif') # Nicer font
rc('font', size=20)                   # Larger font size for labels

# Setup
A few new modules we'll be using...

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import soundfile as sf
from matplotlib import animation, rc
from scipy import signal
import numpy.fft as fft
# import pickle    # NEW: For serializing objects & data

rc('animation', html='jshtml')

path = '/content/drive/My Drive/eces435-work/class6.1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'ncc1864'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Helper functions

`myPlot()`

A quick time-domain signal plot function with my default figure settings and a time x-axis (in seconds).
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `N=#` Number of samples to plot (default: length of signal)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Plot format string (default: none)
  * *New* `x_lim=#` or `x_lim=[x1,x2]` Specify the x-axis limit(s) of the plot
  * *New* `y_lim=#` or `y_lim=[y1,y2]` Specificy the y-axis limit(s) of the plot

In [ ]:
def myPlot(sig, N=0, fs=44100, fig_size=(16,4), x_ax=True, y_ax=True, lw=1, fmt='', x_lim=0, y_lim=0):
  if N==0:
    N = len(sig)

  fig = plt.figure(figsize=fig_size)
  t = np.arange(N)/fs

  plt.plot( t[:N], sig[:N], fmt, linewidth=lw)

  plt.xlabel('Time (sec)')
  ax = plt.gca()    # gca(): "Get current axis", the graph object that's currently plotted
  
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = N/fs   # End of input signal
    else:
      x_lim2 = x_lim
    plt.xlim(0, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim != 0:
      plt.ylim(-y_lim, y_lim)
  else:
    plt.ylim(y_lim)

  fig.tight_layout()
  #plt.ion()
  plt.close()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`myPlotFFT()`

Plot the magnitude frequency response (in dB FS) of a signal with my default figure settings and a frequency x-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `n_fft=#` The size of FFT to use (default: length of input signal)
  * `n_win=#` The length of window to use (default: length of input)
  * `win='hann'` The type of window to use (default: `hann`, or `rect`)
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `x_lim=# or (#,#)` Frequency axis limits (max or range, in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x_ax=True/False` Show x-axis (default: True)
  * `y_ax=True/False` Show y-axis (default: True)
  * `lw=#` Change linewdith of signal (default: 1)
  * `fmt='...'` Change plot formatting (default: none)

In [ ]:
def myPlotFFT(sig, fs=44100, n_fft=0, n_win=0, win='hann', neg_f=False, x_lim=0, y_lim=0, fig_size=(16,6),x_ax=True, y_ax=True, lw=1, fmt=''):
  if n_fft==0:
    n_fft = len(sig)
  if n_win==0:
    n_win = len(sig)

  if win=='hann':  
    win = np.hanning(n_win)
    win_scale = 2
  else:
    win = np.ones(n_win)
    win_scale = 1

  S = np.fft.fft(sig * win, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  if neg_f:
    f = f - (fs/2)
    S = np.fft.fftshift(S)

  S_mag = 2*win_scale*np.abs(S) / n_win     # Frequency magnitude, normalized by length
                                            #    x2 because cos(w) = 0.5e^jw + 0.5e^-jw
                                            #    x2 for Hann because window has 0.5 average

  S_mag += 1e-15                  # Add a small offset to avoid log(0) errors
  S_dBFS = 20*np.log10(S_mag)     # Freq. magnitude in dB full scale (dB FS):
                                  #    cos(w) -> 0 dBFS peak at w


  fig = plt.figure(figsize=fig_size)
  plt.plot(f, S_dBFS, fmt, linewidth=lw) 
  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim2 = fs/2
    if neg_f:
      x_lim = -fs/2
    plt.xlim(x_lim, x_lim2)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim < 0:
      plt.ylim(y_lim, 0)
    elif y_lim > 0:
      plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB FS)')

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # Returning the figure causes issues with interactive matplotlib
  return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

`mySpectrogram()`

A simple wrapper to compute and plot the spectrogram of a signal with my default figure settings, a time x-axis (in seconds) and a frequency y-axis (in Hz), based on the Nyquist rate.
* Required arguments:
  * `sig` Input signal (first argument)
* Optional arguments:
  * `fs=#` Sample rate of signal (default: 44100 Hz)
  * `win='window_name'` The type of analysis window to use (default: 'hann')
  * `n_win=#` The length of window to use per frame (default: 1024)
  * `n_fft=#` The size of FFT to use (default: 1024)
  * `x_lim=# or (#,#)` x-axis limit or range (in seconds)
  * `y_lim=# or (#,#)` y-axis limit or range (in Hz)
  * `fig_size=(W,H)` Change figure dimensions (width, height)
  * `x-ax=True/False` Show x-axis (default: True)
  * `y-ax=True/False` Show y-axis (default: True)

In [ ]:
def mySpectrogram(sig, fs=44100, win='hann', n_win=1024, olap=512, n_fft=1024, x_lim=0, y_lim=0, fig_size=(12,6), x_ax=True, y_ax=True):
  f1, t1, Sxx = signal.stft(sig, fs, window=win, nperseg=n_win, noverlap=olap, nfft=n_fft)

  fig = plt.figure(figsize=fig_size)

  S_mag = 4*np.abs(Sxx) + 1e-15
  S_dBFS = 20*np.log10(S_mag)
  
  plt.pcolormesh(t1, f1, S_dBFS)
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')

  if np.isscalar(x_lim):
    if x_lim == 0:
      x_lim = len(sig) / fs
    plt.xlim(0, x_lim)
  else:
    plt.xlim(x_lim)

  if np.isscalar(y_lim):
    if y_lim == 0:
      y_lim = fs/2
    plt.ylim(0, y_lim)
  else:
    plt.ylim(y_lim)

  ax = plt.gca()
  if x_ax == False:
    ax.xaxis.set_visible(False)
  if y_ax == False:
    ax.yaxis.set_visible(False)
  fig.tight_layout()

  # plt.ion()
  
  # Returning the figure causes issues with interactive matplotlib
  # return fig
  # For saving the figure, use the interactive buton, instead.
  # For further customization and command-line saving, more changes are required.

# Load audio sample

* Go, Phillies!

In [ ]:
x, fs44 = sf.read(path + 'GoPhils.wav')
ipd.Audio(x, rate=fs44)

In [ ]:
myPlot(x)

# Revisiting STFT data requirements

In [ ]:
f1, t1, X = signal.stft(x, fs44, nperseg=2048, noverlap=0, nfft=2048, window='boxcar')

S_mag = np.abs(X) + 1e-15
S_dB = 20*np.log10(S_mag)

plt.pcolormesh(t1, f1, S_dB)
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')
plt.xlim(0,5)
# plt.ylim(0,15000)

In [ ]:
n_freqs, n_frames = np.shape(X)
print('Frequency bins: ', n_freqs)
print('Time frames: ', n_frames)

In [ ]:
n_freqs * n_frames

In [ ]:
len(x)

In [ ]:
t2, x2 = signal.istft(X, fs44, nperseg=2048, noverlap=0, nfft=2048, window='boxcar')
myPlot(x2)

In [ ]:
remove_idx = np.where(S_dB < -75)
X_c = np.copy(X)
X_c[remove_idx] = 0

plt.pcolormesh(t1, f1, 20*np.log10(np.abs(X_c)+1e-15))
plt.ylabel('Frequency (Hz)')
plt.xlabel('Time (sec)')

In [ ]:
t_c, x_c = signal.istft(X_c, fs44, nperseg=2048, noverlap=0, nfft=2048, window='boxcar')

In [ ]:
myPlot(x_c)

In [ ]:
ipd.Audio(x_c, rate=fs44)

In [ ]:
n1,n2 = np.shape(remove_idx)
print('Size (bytes): ', (n_freqs*n_frames - n2) * 4)

# Previously: Filtering = Multiplying in the frequency-domain
  * $H_{id} = 1$, $f \leq f_c$
  * $H_{id} = 0$, $f > f_c$

In [ ]:
L = 512

H_id = np.zeros(L)
f_c = 1000              # in Hz
n_c = f_c/fs44 * L  # Convert Hz to a DFT index
print(n_c)

H_id[:int(n_c)] = 1
H_id[-int(n_c):] = 1

f = fs44 * np.arange(L) / L
plt.plot(f, H_id)
plt.xlabel('Frequency (Hz)')

In [ ]:
h_id = fft.ifft(H_id)
h_id = fft.fftshift(h_id)
plt.plot(h_id)

In [ ]:
x1 = x[:512]
x1z = np.zeros(1024)
x1z[:512] = x1
plt.plot(x1z)

In [ ]:
y = signal.convolve(x1, h_id)
plt.plot(y)

In [ ]:
y1 = signal.lfilter(h_id, 1, x1z)
plt.plot(y1)

In [ ]:
fig1 = myPlotFFT(h_id)
# myPlotFFT(h_id, n_fft=2048)

## High-pass filter

In [ ]:
H_hp = np.zeros(L)
f_c2 = 4000
n_c2 = f_c2/fs44 * L

H_hp[int(n_c2):-int(n_c2)] = 1

f = fs44 * np.arange(L) / L
plt.plot(f, H_hp)
plt.xlabel('Frequency (Hz)')

In [ ]:
h_hp = fft.ifft(H_hp)
h_hp = fft.fftshift(h_hp)
plt.plot(h_hp)

In [ ]:
y_hp = signal.lfilter(h_hp, 1, x1z)
plt.plot(y_hp)

## Band-pass filter

In [ ]:
L = 512

H_bp = np.zeros(L)
f_c1 = 2000              # in Hz
n_c1 = f_c1/fs44 * L  # Convert Hz to a DFT index

f_c2 = 8000
n_c2 = f_c2/fs44 * L

H_bp[int(n_c1):int(n_c2)] = 1
H_bp[-int(n_c2):-int(n_c1)] = 1

f = fs44 * np.arange(L) / L
plt.plot(f, H_bp)
plt.xlabel('Frequency (Hz)')

In [ ]:
h_bp = fft.ifft(H_bp)
h_bp = fft.fftshift(h_bp)
plt.plot(h_bp)

In [ ]:
y_bp = signal.lfilter(h_bp, 1, x1z)
plt.plot(y_bp)

# FIR filter design: *window* method

In [ ]:
N = 128
f_c = 0.15
A_lp = signal.firwin(N, f_c)
plt.plot(A_lp)

In [ ]:
fig = myPlotFFT(A_lp, n_fft=2048)

In [ ]:
y_lp = signal.lfilter(A_lp, 1, x)
ipd.Audio(y_lp, rate=fs44)

In [ ]:
N = 127
f_c = 0.25
A_hp = signal.firwin(N, f_c, pass_zero='highpass')
plt.plot(A_hp)

In [ ]:
fig = myPlotFFT(A_hp, n_fft=2048)

In [ ]:
y_hp = signal.lfilter(A_hp, 1, x)
ipd.Audio(y_hp, rate=fs44)

In [ ]:
N = 128
f_c = [0.1, 0.4]
A_bp = signal.firwin(N, f_c, pass_zero='bandpass')
plt.plot(A_bp)

In [ ]:
fig = myPlotFFT(A_bp, n_fft=2048)

In [ ]:
y_bp = signal.lfilter(A_bp, 1, x)
ipd.Audio(y_bp, rate=fs44)